The below cell is what I used to download the images off the CSV. I filtered out for non-ASCII characters.
Note: I did some preprocessing on the CSV to remove all the not-valid URLs (there were only 3 in initial CSV).

The below code is commented out so that it doesn't interfere with Jupyter notebook running.

In [7]:
'''
import urllib.request
import csv
import os
from urllib.parse import urlparse
import shutil

csv_directory = 'C:/Users/usaid/Desktop/FashionTrends/'
image_dir = 'C:/Users/usaid/Desktop/FashionTrends/images/'

with open(csv_directory + 'image_url.csv', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        if row[1] != 'image url':
            try:
                os.path.basename(urlparse(row[1]).path).encode('ascii')
            except UnicodeEncodeError:
                pass  # string is not ascii
            else:
                with urllib.request.urlopen(row[1]) as response, open(image_dir + os.path.basename(urlparse(row[1]).path), 'wb') as out_file:
                    shutil.copyfileobj(response, out_file)
                    print(os.path.basename(urlparse(row[1]).path))
                pass  # string is ascii
'''

"\nimport urllib.request\nimport csv\nimport os\nfrom urllib.parse import urlparse\nimport shutil\n\ncsv_directory = 'C:/Users/usaid/Desktop/FashionTrends/'\nimage_dir = 'C:/Users/usaid/Desktop/FashionTrends/images/'\n\nwith open(csv_directory + 'image_url.csv', newline='') as file:\n    reader = csv.reader(file, delimiter=',')\n    for row in reader:\n        if row[1] != 'image url':\n            try:\n                os.path.basename(urlparse(row[1]).path).encode('ascii')\n            except UnicodeEncodeError:\n                pass  # string is not ascii\n            else:\n                with urllib.request.urlopen(row[1]) as response, open(image_dir + os.path.basename(urlparse(row[1]).path), 'wb') as out_file:\n                    shutil.copyfileobj(response, out_file)\n                    print(os.path.basename(urlparse(row[1]).path))\n                pass  # string is ascii\n"

I used DeepLabv3 for image segmentation.
https://github.com/bonlime/keras-deeplab-v3-plus
This needs to be downloaded to the same folder the code is running in (the library comes with pre-trained weights, so no installation stuff besides the prerequisite Keras/Tensorflow).

In [8]:
# To Download Keras-DeepLab-v3
#!git clone https://github.com/bonlime/keras-deeplab-v3-plus

In [9]:
import cv2
import glob
from model import Deeplabv3
import numpy as np
import matplotlib.pyplot as plt
import os

#Put in your directory where images are located.
image_dir = "C:/Users/usaid/Desktop/FashionTrends/images/ChicImages_1_75/"
images = glob.glob(image_dir + "*.jpg")

deeplab_model = Deeplabv3()
print("Initialized Pipeline")
#fig, ax = plt.subplots(6, 3);
i = 0
for img in images:
    image = plt.imread(img)
    w, h, _ = image.shape
    ratio = 512. / np.max([w,h])
    resized_original = cv2.resize(image,(int(ratio*h),int(ratio*w)))
    resized = resized_original / 127.5 - 1.
    pad_x = int(512 - resized.shape[0])
    pad_y = int(512 - resized.shape[1])
    resized2 = np.pad(resized,((0,pad_x),(0,pad_y),(0,0)),mode='constant')
    res = deeplab_model.predict(np.expand_dims(resized2,0))
    labels = np.argmax(res.squeeze(),-1)
    np.place(labels, labels != 15, [0])
    masked_image = cv2.bitwise_and(resized_original, resized_original, mask=labels[:512 - pad_x, :512 - pad_y].astype(np.uint8))
    #ax[i, 0].imshow(labels[:512 - pad_x, :512 - pad_y])
    #ax[i, 1].imshow(image)
    #ax[i, 2].imshow(masked_image)
    cv2.cvtColor(masked_image, cv2.COLOR_RGB2BGR, masked_image)
    
    # The image_dir + out/ directory needs to already be there before this works
    cv2.imwrite(image_dir + "out/" + os.path.basename(img), masked_image)
    i += 1
print("Finished")
#plt.show()

Initialized Pipeline
Finished
